In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from matplotlib.patches import Ellipse
from sklearn.model_selection import train_test_split
import tensorflow as tf
import os

ds = tf.contrib.distributions
k = tf.keras

font = {'size'   : 14}

matplotlib.rc('font', **font)

In [ ]:
r_dist = tf.placeholder_with_default(2., shape=())

bkg = ds.MultivariateNormalFullCovariance(loc=[r_dist, 0.],
        covariance_matrix=[[5.,0.],[0.,9.]], name="p0")

sig = ds.MultivariateNormalDiag(loc=[0., 0.],
        scale_diag=[1., 1.], name="p1")

mu = tf.placeholder(shape=(), dtype=tf.float32, name="mu")
mix = ds.Mixture(cat=ds.Categorical(probs=[1.-mu, mu]),
               components = [bkg, sig])

In [ ]:
# generate training data and split validation hold_out
n_samples_train = 125000
X_sample_tensors = {}
y_values = {}
p0_sample = "p0_sample"
X_sample_tensors[p0_sample] = bkg.sample(n_samples_train // 2, seed=7, name="sig_sample") 
y_values[p0_sample] = 0.
p1_sample = "p1_sample"
X_sample_tensors[p1_sample] = sig.sample(n_samples_train // 2, seed=17, name="bkg_sample") 
y_values[p1_sample] = 1.

samples = {}

with tf.Session() as sess:
  for name, sample_tensor in X_sample_tensors.items():
      samples[name] = {}
      samples[name]["X"] = sess.run(sample_tensor)
      samples[name]["y"] =  y_values[name]*np.ones(sample_tensor.shape[0],
                                                     dtype=np.float32)
train_samples = {}
valid_samples = {}

for name,sample in samples.items():
  keys = sample.keys()
  split_sample = train_test_split(*list(sample.values()),
                                  test_size = 0.4,
                                  random_state=17)
  train_samples[name] = { k : split_sample[i] for k,i in zip(keys,[0,2])}
  valid_samples[name] = { k : split_sample[i] for k,i in zip(keys,[1,3])}

In [ ]:
# generate test dataset
n_samples_test = 500000
X_sample_tensors = {}
y_values = {}
p0_sample = "p0_sample"
X_sample_tensors[p0_sample] = bkg.sample(n_samples_test // 2, seed=45, name="sig_sample") 
y_values[p0_sample] = 0.
p1_sample = "p1_sample"
X_sample_tensors[p1_sample] = sig.sample(n_samples_test // 2, seed=79, name="bkg_sample") 
y_values[p1_sample] = 1.

test_samples = {}

with tf.Session() as sess:
  for name, sample_tensor in X_sample_tensors.items():
    test_samples[name] = {}
    test_samples[name]["X"] = sess.run(sample_tensor)
    test_samples[name]["y"] =  y_values[name]*np.ones(sample_tensor.shape[0],
                                                     dtype=np.float32)

In [ ]:
# save the data generated
DATA_DIR = "../data/2d_gaussian/"

s_dirs = {"train" : train_samples,
          "valid" : valid_samples,
          "test" : test_samples}

for split, s_dict in s_dirs.items():
  data_folder = f'../data/2d_gaussian/{split}/'
  if not os.path.exists(data_folder):
    os.mkdir(data_folder)
  for name, sample in s_dict.items():
    file_name = f'{data_folder}{name}.npz'
    np.savez(file_name, **sample)

In [ ]:
x_scan  = np.linspace(-5.,5.,1001, endpoint=True, dtype=np.float32)
y_scan  = np.linspace(-5.,5.,1001, endpoint=True, dtype=np.float32)

X_grid = np.stack(np.meshgrid(x_scan, y_scan),axis=-1)
grid_shape = list(X_grid.shape[:-1])
X_flat = np.reshape(X_grid, [-1,2])

In [ ]:
X_grid_t = tf.placeholder(shape=(None, None, 2), dtype=tf.float32)

mix_grid = mix.log_prob(X_grid_t)
bkg_grid = bkg.log_prob(X_grid_t)
sig_grid = sig.log_prob(X_grid_t)
bkg_cov = bkg.covariance()
sig_cov = sig.covariance()
bkg_mean = bkg.mean()
sig_mean = sig.mean()

s = 40.
b = 960.
nu = 1.
mu_val = (nu*s)/(nu*s+b)
print(mu_val)

with tf.Session() as sess:
    mix_grid_arr = sess.run(mix_grid, feed_dict={X_grid_t : X_grid, mu : mu_val})
    bkg_grid_arr = sess.run(bkg_grid, feed_dict={X_grid_t : X_grid})
    sig_grid_arr =  sess.run(sig_grid, feed_dict={X_grid_t : X_grid})
    bkg_cov_arr = sess.run(bkg_cov)
    sig_cov_arr = sess.run(sig_cov)
    bkg_mean_arr = sess.run(bkg_mean)
    sig_mean_arr = sess.run(sig_mean)

lr_mix_bkg = mix_grid_arr - bkg_grid_arr
lr_sig_bkg = sig_grid_arr - bkg_grid_arr

In [ ]:
figsize = (8,6)
ax_xlabel = "variable 1"
ax_ylabel = "variable 2"

In [ ]:
fig, ax = plt.subplots(1,1,figsize=figsize)

levels = np.linspace(-0.05,0.35,9)
con = ax.contourf(x_scan,y_scan, lr_mix_bkg, levels=levels)
fig.colorbar(con, ax=ax)

ax.set_xlabel(ax_xlabel)
ax.set_ylabel(ax_ylabel)
ax.set_aspect('equal')

fig.savefig("lr_hyphotesis.pdf", bbox_inches='tight')
fig

In [ ]:
fig, ax = plt.subplots(1,1,figsize=figsize)

c_levels = np.linspace(-21,3,13)
con = ax.contourf(x_scan,y_scan, lr_sig_bkg, levels=c_levels )
fig.colorbar(con, ax=ax)

for mean, cov, color in [[bkg_mean_arr, bkg_cov_arr, 'blue'],
                         [sig_mean_arr, sig_cov_arr, 'red']]:
  l, v = np.linalg.eig(cov)
  l = np.sqrt(l)
  for j in range(1, 3):
    ell = Ellipse(xy=mean,
                  width=l[0]*j*2, height=l[1]*j*2,
                  angle=np.rad2deg(np.arccos(v[0, 0])))
    ell.set_facecolor('none')
    ell.set_edgecolor(color)
    ell.set_linestyle('--')
    ell.set_linewidth(2)
    ax.add_artist(ell)

ax.set_xlabel(ax_xlabel)
ax.set_ylabel(ax_ylabel)
ax.set_aspect('equal')

fig.savefig("lr_density.pdf", bbox_inches='tight')
fig

In [ ]:
x_scan  = np.linspace(-5.,5.,1001, endpoint=True, dtype=np.float32)
y_scan  = np.linspace(-5.,5.,1001, endpoint=True, dtype=np.float32)
X_grid = np.stack(np.meshgrid(x_scan, y_scan),axis=-1)
grid_shape = list(X_grid.shape[:-1])
X_flat = np.reshape(X_grid, [-1,2])

model_path = "log_dir/cross_entropy_n_bins_2_bs_128_lr_0.01.h5"

model = k.models.load_model(model_path)

y_flat = model.predict_proba(X_flat, batch_size=1024)
y_grid = np.reshape(y_flat, grid_shape+[-1])

approx_lr = np.log(y_grid[:,:,1]) - np.log(y_grid[:,:,0])
model.summary()

In [ ]:
fig, ax = plt.subplots(1,1,figsize=figsize)

c_levels = np.linspace(-21,3,13)
con = ax.contourf(x_scan,y_scan, approx_lr, levels=c_levels )
fig.colorbar(con, ax=ax)

ax.set_xlabel(ax_xlabel)
ax.set_ylabel(ax_ylabel)
ax.set_aspect('equal')

fig.savefig("lr_approx_ratio.pdf", bbox_inches='tight')
fig